# Create a simple tensor with random values

In [2]:
import numpy as np

np.set_printoptions(suppress=True)

# Creating a randomly distributed parameter
params=np.random.uniform(low=-50,high=150,size=20)

# Ensuring important values at begining
params[0]=params.max()+1
params[1]=params.min()-1
params[2]=0

params=np.round(params,2)
print(params)

[144.36 -41.58   0.    84.27 143.36 136.17 -16.87  43.34  15.51 -13.42
   3.74 134.41  31.83  13.78 128.09 115.63  79.32 -40.58 -20.38 121.84]


# Define Quantization method

In [13]:
def clamp(params_q:np.array,lower_bound:int,upper_bound:int)->np.array:
  params_q[params_q < lower_bound]=lower_bound
  params_q[params_q > upper_bound]=upper_bound
  return params_q


def asymmetric_quantization(params:np.array,bits:int)->tuple[np.array,float,int]:
  '''Generally asymmetric quantization is used in-
        1) After Activation function (Most of the values are in fraction and not zero centered)
        2) Input
        3) In the Post training quantization (to handler back propagation error)'''
  # Max value
  alpha=np.max(params)
  # Min value
  beta=np.min(params)
  # Scale - This will tell range of the particular tensor, For shrinking
  scale=(alpha-beta)/(2**bits-1)
  # zero - THis will define the mean or center value
  zero= -1 *np.round(beta/scale)
  lower_bound,upper_bound=0,2**bits-1
  # Quantize the parameter
  quantize=clamp(np.round(params/scale+zero),lower_bound,upper_bound).astype(np.int32)
  return quantize,scale,zero

def asymmetric_dequantize(params_q:np.array,scale:float,zero:int)->np.array:
  return (params_q-zero)*scale

def symmetric_quantization(params:np.array,bits:int)->tuple[np.array,float]:
  '''Generally symmetric quantization is used in -
      1) Weights (They have zero centered value)
      2)Quantization aware training
  '''
  alpha = np.max(np.abs(params))
  scale = alpha / (2**(bits-1)-1)
  lower_bound = -2**(bits-1)
  upper_bound = 2**(bits-1)-1
  # Quantize the parameter
  quantize=clamp(np.round(params/scale),lower_bound,upper_bound).astype(np.int32)
  return quantize,scale

def symmetric_dequantize(params_q:np.array,scale:int)->np.array:
  return params_q*scale

def quantize_error(params_q:np.array,params:np.array):
  # Calsulate MSE
  return np.mean((params-params_q)**2)


(asymmetric_q, asymmetric_scale, asymmetric_zero) = asymmetric_quantization(params, 8)
(symmetric_q, symmetric_scale) = symmetric_quantization(params, 8)

print(f'Original:')
print(np.round(params, 2))
print('')
print(f'Asymmetric scale: {asymmetric_scale}, zero: {asymmetric_zero}')
print(asymmetric_q)
print('')
print(f'Symmetric scale: {symmetric_scale}')
print(symmetric_q)


Original:
[144.36 -41.58   0.    84.27 143.36 136.17 -16.87  43.34  15.51 -13.42
   3.74 134.41  31.83  13.78 128.09 115.63  79.32 -40.58 -20.38 121.84]

Asymmetric scale: 0.7291764705882353, zero: 57.0
[255   0  57 173 254 244  34 116  78  39  62 241 101  76 233 216 166   1
  29 224]

Symmetric scale: 1.1366929133858268
[127 -37   0  74 126 120 -15  38  14 -12   3 118  28  12 113 102  70 -36
 -18 107]


In [14]:
# Dequantize the parameters back to 32 bits
params_deq_asymmetric = asymmetric_dequantize(asymmetric_q, asymmetric_scale, asymmetric_zero)
params_deq_symmetric = symmetric_dequantize(symmetric_q, symmetric_scale)

print(f'Original:')
print(np.round(params, 2))
print('')
print(f'Dequantize Asymmetric:')
print(np.round(params_deq_asymmetric,2))
print('')
print(f'Dequantize Symmetric:')
print(np.round(params_deq_symmetric, 2))

Original:
[144.36 -41.58   0.    84.27 143.36 136.17 -16.87  43.34  15.51 -13.42
   3.74 134.41  31.83  13.78 128.09 115.63  79.32 -40.58 -20.38 121.84]

Dequantize Asymmetric:
[144.38 -41.56   0.    84.58 143.65 136.36 -16.77  43.02  15.31 -13.13
   3.65 134.17  32.08  13.85 128.34 115.94  79.48 -40.83 -20.42 121.77]

Dequantize Symmetric:
[144.36 -42.06   0.    84.12 143.22 136.4  -17.05  43.19  15.91 -13.64
   3.41 134.13  31.83  13.64 128.45 115.94  79.57 -40.92 -20.46 121.63]


In [15]:
# Calculate the quantization error
print(f'{"Asymmetric error: ":>20}{np.round(quantize_error(params, params_deq_asymmetric), 2)}')
print(f'{"Symmetric error: ":>20}{np.round(quantize_error(params, params_deq_symmetric), 2)}')

  Asymmetric error: 0.04
   Symmetric error: 0.06


In [18]:
 ''' Symmetric quantization have range isssue.
     That's why it show more quantization error then asymmetric
     It has forced to be around zero,this cause poor usage of range.
      But we still use it-
          * This gives us simple calculation
          * Faster hardware support  '''

" Symmetric quantization have range isssue.\n    That's why it show more quantization error then asymmetric\n    It has forced to be around zero,this cause poor usage of range.\n     But we still use it-\n         * This gives us simple calculation \n         * Faster hardware support  "